In [1]:
!pip install kaggle

In [2]:
# configuring the path of Kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
# daatset api
!kaggle datasets download -d thepbordin/indoor-object-detection

 98% 359M/367M [00:04<00:00, 104MB/s]
100% 367M/367M [00:04<00:00, 91.0MB/s]


In [4]:
# extracting the compessed Dataset
from zipfile import ZipFile
dataset = '/content/indoor-object-detection.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [5]:
import os
import torch
import torchvision
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F

random_seed = 213
torch.manual_seed(random_seed)

In [6]:
#data_dir  = '/content/train/images'

#classes = os.listdir(data_dir)
classes={0:"door",1:"cabinetDoor",2:"refrigeratorDoor",3:"window",4:"chair",5:"table",
       6:"cabinet",7:"couch",8:"openedDoor",9:"pole"}
print(classes)
print(f"length: {len(classes)}")

{0: 'door', 1: 'cabinetDoor', 2: 'refrigeratorDoor', 3: 'window', 4: 'chair', 5: 'table', 6: 'cabinet', 7: 'couch', 8: 'openedDoor', 9: 'pole'}
length: 10


In [7]:
import os

# Count image files
image_files = os.listdir('train/images')
num_image_files = len(image_files)

# Count label files
label_files = os.listdir('train/labels')
num_label_files = len(label_files)

print(f'Number of image files: {num_image_files}')
print(f'Number of label files: {num_label_files}')

Number of image files: 1012
Number of label files: 1008


In [8]:
image_files = os.listdir('train/images')
label_files = os.listdir('train/labels')

# Remove file extensions
image_names = [os.path.splitext(filename)[0] for filename in image_files]
label_names = [os.path.splitext(filename)[0] for filename in label_files]

print(f'Number of image files: {len(image_names)}')
print(f'Number of label files: {len(label_names)}')


Number of image files: 1012
Number of label files: 1008


In [9]:
extra_images = [image_name for image_name in image_names if image_name not in label_names]


In [10]:
import os

images_directory = 'train/images'

for image_name in extra_images:
    image_file = os.path.join(images_directory, f'{image_name}.jpg')  # Adjust the file extension as needed
    os.remove(image_file)


In [13]:
image_files = os.listdir('train/images')
label_files = os.listdir('train/labels')

# Remove file extensions
image_names = [os.path.splitext(filename)[0] for filename in image_files]
label_names = [os.path.splitext(filename)[0] for filename in label_files]

print(f'Number of image files: {len(image_names)}')
print(f'Number of label files: {len(label_names)}')


Number of image files: 1008
Number of label files: 1008


In [19]:
import os
import glob
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Conv2D, Dense, Input, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define the directory paths
train_images_directory = 'train/images'
train_labels_directory = 'train/labels'

# Load and preprocess labels from text files
def load_labels(directory):
    labels = []

    # Iterate through all text files in the directory
    for filename in glob.glob(os.path.join(directory, '*.txt')):
        with open(filename, 'r') as file:
            lines = file.readlines()

        annotations = []
        for line in lines:
            parts = line.strip().split(' ')
            class_label = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])

            annotation = {
                'class_label': class_label,
                'bounding_box': (x_center, y_center, width, height)
            }

            annotations.append(annotation)

        labels.append(annotations)

    return labels

# Load image filenames
#image_files = glob.glob(os.path.join(train_images_directory, '*.jpg'))

# Load EfficientDet model
def efficientdet_model(num_classes):
    base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    base_model.trainable = False
    x = base_model.output
    x = Conv2D(256, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    predictions = Dense(num_classes, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# Create an instance of the EfficientDet model
num_classes = 10  # Replace with the actual number of classes
model = efficientdet_model(num_classes)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Load and preprocess labels
train_labels = load_labels(train_labels_directory)

print(len(train_labels))
print(len(image_files))

# Ensure that the number of samples in labels matches the number of samples in image_files
assert len(train_labels) == len(image_files), "Number of labels and images must match."

# Load and preprocess images and labels
X_train = []  # To store preprocessed images
Y_train = []  # To store corresponding labels

for image_file, labels in (image_files, train_labels):
    # Load and preprocess the image
    img = load_img(image_file, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0  # Normalize the image
    X_train.append(img_array)

    # Create a label vector for this image
    label_vector = [0] * num_classes
    for annotation in labels:
        class_label = annotation['class_label']
        label_vector[class_label] = 1
    Y_train.append(label_vector)

X_train = np.array(X_train)
Y_train = np.array(Y_train)

# Train the model
model.fit(X_train, Y_train, epochs=10)


1008
1008


ValueError: ignored

In [17]:
import os
import glob
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Conv2D, Dense, Input, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define the directory paths
train_images_directory = 'train/images'
train_labels_directory = 'train/labels'

# Load and preprocess labels from text files
def load_labels(directory):
    labels = []

    # Iterate through all text files in the directory
    for filename in glob.glob(os.path.join(directory, '*.txt')):
        with open(filename, 'r') as file:
            lines = file.readlines()

        annotations = []
        for line in lines:
            parts = line.strip().split(' ')
            class_label = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])

            annotation = {
                'class_label': class_label,
                'bounding_box': (x_center, y_center, width, height)
            }

            annotations.append(annotation)

        labels.append(annotations)

    return labels

# Load and preprocess image data using ImageDataGenerator
image_size = (224, 224)
batch_size = 32

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_images_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode=None,  # Set to None for custom labels
    shuffle=False  # Important: Do not shuffle images for correct label matching
)

# Load EfficientDet model
def efficientdet_model(num_classes):
    base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    base_model.trainable = False
    x = base_model.output
    x = Conv2D(256, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    predictions = Dense(num_classes, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# Create an instance of the EfficientDet model
num_classes = 10  # Replace with the actual number of classes
model = efficientdet_model(num_classes)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Load and preprocess labels
train_labels = load_labels(train_labels_directory)

print(len(train_labels))
print(len(image_files))
print(len(train_generator.filenames))
# Ensure that the number of samples in labels matches the number of samples in the generator
#assert len(train_labels) == len(train_generator.filenames), "Number of labels and images must match."

# Train the model
model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=10)



Found 0 images belonging to 0 classes.
1008
1008
0


ValueError: ignored

In [ ]:
train_images='/content/train/images'
val_images='/content/valid/images'
test_images='/content/test/images'

train_labels='/content/train/labels'
val_labels='/content/valid/labels'
test_labels='/content/test/labels'

In [ ]:
import os
import glob

# Directory containing the annotation text files
labels_directory = 'train/labels'

# List to store all annotations from all files
train_labels = []

# Iterate through all text files in the directory
for filename in glob.glob(os.path.join(labels_directory, '*.txt')):
    with open(filename, 'r') as file:
        lines = file.readlines()

    annotations = []
    for line in lines:
        parts = line.strip().split(' ')
        class_label = int(parts[0])
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])

        annotation = {
            'class_label': class_label,
            'bounding_box': (x_center, y_center, width, height)
        }

        annotations.append(annotation)

    # Extend the train_labels list with annotations from the current file
    train_labels.extend(annotations)

#


import os
import glob

# Directory containing the annotation text files
labels_directory = 'test/labels'

# List to store all annotations
test_labels = []

# Iterate through all text files in the directory
for filename in glob.glob(os.path.join(labels_directory, '/*.txt')):
    with open(filename, 'r') as file:
        lines = file.readlines()

    annotations = []
    for line in lines:
        parts = line.strip().split(' ')
        class_label = int(parts[0])
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])

        annotation = {
            'class_label': class_label,
            'bounding_box': (x_center, y_center, width, height)
        }

        annotations.append(annotation)

    test_labels.extend(annotations)

# Now, train_labels is a single list containing all annotations from all text files.


import os
import glob

# Directory containing the annotation text files
labels_directory = 'valid/labels'

# List to store all annotations
val_labels = []

# Iterate through all text files in the directory
for filename in glob.glob(os.path.join(labels_directory, '*.txt')):
    with open(filename, 'r') as file:
        lines = file.readlines()

    annotations = []
    for line in lines:
        parts = line.strip().split(' ')
        class_label = int(parts[0])
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])

        annotation = {
            'class_label': class_label,
            'bounding_box': (x_center, y_center, width, height)
        }

        annotations.append(annotation)

    val_labels.extend(annotations)

# Now, train_labels is a single list containing all annotations from all text files.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data generators for train, validation, and test sets
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_directory('train/images', target_size=(224, 224), batch_size=32, class_mode=None)
val_generator = val_datagen.flow_from_directory('valid/images', target_size=(224, 224), batch_size=32, class_mode=None)
test_generator = test_datagen.flow_from_directory('test/images', target_size=(224, 224), batch_size=32, class_mode=None)

# Train the model
model.fit(train_generator, epochs=10, validation_data=val_generator)

# Evaluate the model
loss, accuracy = model.evaluate(test_generator)

# Perform object detection and localization on a test image
image = load_image('test.jpg')  # Implement load_image function to load and preprocess an image
predictions = model.predict(image)


In [ ]:
# Import the necessary libraries
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Conv2D, Dense, Input, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define the EfficientDet architecture
def efficientdet_model():
    # Use EfficientNetB0 as the base model
    base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

    # Freeze the base model layers
    base_model.trainable = False

    # Add additional layers for object detection
    x = base_model.output
    x = Conv2D(256, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    predictions = Dense(10, activation='sigmoid')(x)

    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

# Create an instance of the EfficientDet model
model = efficientdet_model()

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(val_images, val_labels))

# Evaluate the model
loss, accuracy = model.evaluate(test_images, test_labels)

# Perform object detection and localization on a test image
image = load_image('test.jpg')
predictions = model.predict(image)